# Classes RelationalQueryProcessor and TriplestoreQueryProcessor
Add the following method to both the classes:

## searchByAuthor(name : str) : DataFrame
It returns a data frame with all the publications (i.e. the rows) that have been authored by people having the input string included either in their family name or given name. The match must be done lowercase and can be also partial, e.g. the input string “jo" will match with “Jo", “John", “Johnathan" and “Guido".

# Class GenericQueryProcessor
Add the following method to the class:

## searchByAuthor(name : str) : list[Publication]
It returns a list of Publication objects referring to all the publications (i.e. the rows) that have been authored by people having the input string included either in their family name or given name. The match must be done lowercase and can be also partial, e.g. the input string “jo" will match with “Jo", “John", “Johnathan" and “Guido".


I will define my data model using Python classes

hello
